In [1]:
# Pytorch libraries
import torch
import torch.nn.functional as F
# classic libraries
import numpy as np

import errno
import os
import shutil
import sys



def create_folder(folder_path):
    """Create a folder if it does not exist.
    """
    try:
        os.makedirs(folder_path)
    except OSError as _e:
        if _e.errno != errno.EEXIST:
            raise


def to_np(var):
    """Exports torch.Tensor to Numpy array.
    """
    return var.detach().cpu().numpy()

def clear_folder(folder_path):
    """Clear all contents recursively if the folder exists.
    Create the folder if it has been accidently deleted.
    """
    create_folder(folder_path)
    for the_file in os.listdir(folder_path):
        _file_path = os.path.join(folder_path, the_file)
        try:
            if os.path.isfile(_file_path):
                os.unlink(_file_path)
            elif os.path.isdir(_file_path):
                shutil.rmtree(_file_path)
        except OSError as _e:
            print(_e)

In [2]:
# Pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
# classic libraries
import numpy as np
import os
import sys

CUDA = True
data_dir = '../dataset/fashionmnist'
out_dir = '../output'
epochs=3
#LOG_FILE = os.path.join(OUT_PATH, 'log.txt')
batch_size = 128
lr = 0.0002
latent_dim=100
classes = 10
img_size = 64
log_interval=100
channels=1
train=True
seed = 1

class Generator(nn.Module):
    def __init__(self, classes, channels, img_size, latent_dim):
        super(Generator, self).__init__()
        self.classes = classes
        self.channels = channels
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.img_shape = (self.channels, self.img_size, self.img_size)
        self.label_embedding = nn.Embedding(self.classes, self.classes)

        self.model = nn.Sequential(
            *self._create_layer(self.latent_dim + self.classes, 128, False),
            *self._create_layer(128, 256),
            *self._create_layer(256, 512),
            *self._create_layer(512, 1024),
            nn.Linear(1024, int(np.prod(self.img_shape))),
            nn.Tanh()
        )

    def _create_layer(self, size_in, size_out, normalize=True):
        layers = [nn.Linear(size_in, size_out)]
        if normalize:
            layers.append(nn.BatchNorm1d(size_out))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        return layers

    def forward(self, noise, labels):
        z = torch.cat((self.label_embedding(labels), noise), -1)
        x = self.model(z)
        x = x.view(x.size(0), *self.img_shape)
        return x


class Discriminator(nn.Module):
    def __init__(self, classes, channels, img_size, latent_dim):
        super(Discriminator, self).__init__()
        self.classes = classes
        self.channels = channels
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.img_shape = (self.channels, self.img_size, self.img_size)
        self.label_embedding = nn.Embedding(self.classes, self.classes)
        self.adv_loss = torch.nn.BCELoss()

        self.model = nn.Sequential(
            *self._create_layer(self.classes + int(np.prod(self.img_shape)), 1024, False, True),
            *self._create_layer(1024, 512, True, True),
            *self._create_layer(512, 256, True, True),
            *self._create_layer(256, 128, False, False),
            *self._create_layer(128, 1, False, False),
            nn.Sigmoid()
        )

    def _create_layer(self, size_in, size_out, drop_out=True, act_func=True):
        layers = [nn.Linear(size_in, size_out)]
        if drop_out:
            layers.append(nn.Dropout(0.4))
        if act_func:
            layers.append(nn.LeakyReLU(0.2, inplace=True))
        return layers

    def forward(self, image, labels):
        x = torch.cat((image.view(image.size(0), -1), self.label_embedding(labels)), -1)
        return self.model(x)

    def loss(self, output, label):
        return self.adv_loss(output, label)


In [7]:
# Pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
# classic libraries
import numpy as np
#import statsmodels.api as sm    # to estimate an average with 'loess'
import matplotlib.pyplot as plt
#from matplotlib import rc
#rc('font', **font)
#rc('text', usetex=True)
import pandas as pd
import random, string
import os, time, datetime, json
# perso libraries
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 14}
#rc('font', **font)
#rc('text', usetex=True)
import pandas as pd
import random, string
import os, time, datetime, json
#-------------------------------------------------#
#               A) Hyper-parameters               #
#-------------------------------------------------#
CUDA = True
data_dir = '../dataset/fashionmnist'
out_dir = '../output'
epochs=30
#LOG_FILE = os.path.join(OUT_PATH, 'log.txt')
batch_size = 128
lr = 0.0002
latent_dim=100
classes = 10
img_size = 64
log_interval=100
channels=1
train=True
seed = 1
#-------------------------------------------------#
#               B) Data/Model/Loss                #
#-------------------------------------------------#
# B.1) dataset/corpus
#--------------------
# Corpus
# print the number of parameters


        
clear_folder(out_dir)
#print("Logging to {}\n".format(LOG_FILE))
#sys.stdout = utils.StdOut(LOG_FILE)
CUDA = CUDA and torch.cuda.is_available()
print("PyTorch version: {}".format(torch.__version__))
if CUDA:
    print("CUDA version: {}\n".format(torch.version.cuda))
if seed is None:
    seed = np.random.randint(1, 10000)
print("Random Seed: ", seed)
np.random.seed(seed)
torch.manual_seed(seed)
if CUDA:
    torch.cuda.manual_seed(seed)
cudnn.benchmark = True
device = torch.device("cuda:0" if CUDA else "cpu")

def _to_onehot(var, dim):
        res = torch.zeros((var.shape[0], dim), device=device)
        res[range(var.shape[0]), var] = 1.
        return res

def save_to(name=None,
                verbose=True):
        if verbose:
            print('\nSaving models to {}_G.pt and {}_D.pt ...'.format(name, name))
        torch.save(netG.state_dict(), os.path.join(out_dir, '{}_G.pt'.format(name)))
        torch.save(netD.state_dict(), os.path.join(out_dir, '{}_D.pt'.format(name)))


dataset = dset.FashionMNIST(root=data_dir, download=True,
                     transform=transforms.Compose([
                     transforms.Resize(img_size),
                     transforms.ToTensor(),
                     transforms.Normalize((0.5,), (0.5,))
                     ]))

assert dataset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=16, pin_memory=True)

netG = Generator( classes, channels, img_size, latent_dim).to(device)
#netG.apply(weights_init)
print(netG)
nbr_param = sum(p.numel() for p in netG.parameters() if p.requires_grad)
print("generator params", nbr_param)

netD = Discriminator(classes, channels, img_size, latent_dim).to(device)
#netD.apply(weights_init)
print(netD)
nbr_param = sum(p.numel() for p in netD.parameters() if p.requires_grad)
print("discrimator params", nbr_param)

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))




netG.train()
netD.train()
viz_z = torch.zeros((batch_size, latent_dim), device=device)
viz_noise = torch.randn(dataloader.batch_size, latent_dim, device=device)
nrows = dataloader.batch_size // 8
viz_label = torch.LongTensor(np.array([num for _ in range(nrows) for num in range(8)])).to(device)
viz_onehot = _to_onehot(viz_label, dim=classes)
total_time = time.time()
for epoch in range(epochs):
    batch_time = time.time()
    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)
        batch_size = data.size(0)
        real_label = torch.full((batch_size, 1), 1., device=device)
        fake_label = torch.full((batch_size, 1), 0., device=device)
        # Train G
        netG.zero_grad()
        z_noise = torch.randn(batch_size, latent_dim, device=device)
        x_fake_labels = torch.randint(0, classes, (batch_size,), device=device)
        x_fake = netG(z_noise, x_fake_labels)
        y_fake_g = netD(x_fake, x_fake_labels)
        g_loss = netD.loss(y_fake_g, real_label)
        g_loss.backward()
        optimizerG.step()

        # Train D
        netD.zero_grad()
        y_real = netD(data, target)
        d_real_loss = netD.loss(y_real, real_label)

        y_fake_d = netD(x_fake.detach(), x_fake_labels)
        d_fake_loss = netD.loss(y_fake_d, fake_label)
        d_loss = (d_real_loss + d_fake_loss) / 2
        d_loss.backward()
        optimizerD.step()

        if batch_idx % log_interval == 0 and batch_idx > 0:  
            print('Epoch {} [{}/{}] loss_D: {:.4f} loss_G: {:.4f} time: {:.2f}'.format(
                  epoch, batch_idx, len(dataloader),
                  d_loss.mean().item(),
                  g_loss.mean().item(),
                  time.time() - batch_time))
            vutils.save_image(data, os.path.join(out_dir, 'real_samples.png'), normalize=True)
            with torch.no_grad():
                viz_sample = netG(viz_noise, viz_label)
                vutils.save_image(viz_sample, os.path.join(out_dir, 'fake_samples_{}.png'.format(epoch)), nrow=8, normalize=True)
            batch_time = time.time()

    save_to( name='CGAN_{}'.format(epoch))
print('Total train time: {:.2f}'.format(time.time() - total_time))
    


PyTorch version: 1.4.0
CUDA version: 10.1

Random Seed:  1
Generator(
  (label_embedding): Embedding(10, 10)
  (model): Sequential(
    (0): Linear(in_features=110, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Linear(in_features=1024, out_features=4096, bias=True)
    (12): Tanh()
  )
)
generator params 4906212
Discriminator(
  (labe

In [9]:
print('hi')

hi


In [24]:
import os
import sys

from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torchvision.utils as vutils

from scipy.interpolate import interp1d

VIZ_MODE=0
CUDA = True
data_dir = 'dataset/fashionmnist'
class_dir = 'classes/9'
out_dir = 'output/'
epochs=3
#LOG_FILE = os.path.join(OUT_PATH, 'log.txt')
batch_size = 1
lr = 0.0002
latent_dim=100
classes = 10
img_size = 28
log_interval=100
channels=1
train=True
seed = 1

#print("Logging to {}\n".format(LOG_FILE))
#sys.stdout = utils.StdOut(LOG_FILE)
CUDA = CUDA and torch.cuda.is_available()
print("PyTorch version: {}".format(torch.__version__))
if CUDA:
    print("CUDA version: {}\n".format(torch.version.cuda))

if seed is None:
    seed = np.random.randint(1, 10000)
print("Random Seed: ", seed)
np.random.seed(seed)
torch.manual_seed(seed)
if CUDA:
    torch.cuda.manual_seed(seed)
cudnn.benchmark = True      # May train faster but cost more memory

device = torch.device("cuda:0" if CUDA else "cpu")
netG = Generator(classes, channels, img_size, latent_dim).to(device)
netG.load_state_dict(torch.load(os.path.join(out_dir, 'CGAN_199_G.pt'),map_location=torch.device('cpu')))
netG.to(device)
netD = Discriminator(classes, channels, img_size, latent_dim).to(device)

netG.eval()
netD.eval()
if batch_size is None:
    batch_size = data_loader.batch_size
nrows = batch_size // 1
if nrows == 0:
    nrows =1
for i in range(50):
    viz_labels = np.array([num for _ in range(nrows) for num in range(9,10)])
    viz_labels = torch.LongTensor(viz_labels).to(device)
    # viz_label = torch.LongTensor(np.array([num for _ in range(nrows) for num in range(8)])).to(device)
    # viz_onehot = _to_onehot(viz_label, dim=classes)

    with torch.no_grad():
        # z_noise = torch.randn(batch_size, latent_dim, device=device)
        # x_fake_labels = torch.randint(0, classes, (batch_size,), device=device)
        # x_fake = netG(z_noise, x_fake_labels)
        viz_tensor = torch.randn(batch_size, latent_dim, device=device)
        viz_sample = netG(viz_tensor, viz_labels)
        viz_vector = to_np(viz_tensor).reshape(batch_size, latent_dim)
        cur_time = datetime.now().strftime("%Y%m%d-%H%M%S")
        np.savetxt('vec_{}.txt'.format(cur_time), viz_vector)
        vutils.save_image(viz_sample, os.path.join(class_dir,'img_{}_{}.png'.format(cur_time,i)), nrow=1, normalize=True)



PyTorch version: 1.3.1
Random Seed:  1
